# IBM Data Science Professional Certification WikiScraper by Tyler King

In [1]:
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import json
import requests
import numpy as np

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
mainpage=urlopen(url)
mainsoup=BeautifulSoup(mainpage, 'html.parser')
hoodtable= mainsoup.findAll('table')

In [3]:
rows=hoodtable[0].tbody.findAll('tr')

In [4]:

PC=[]
Boro=[]
Hood=[]
for row in rows:
    rowdata=row.findAll('td')
    data=[]
    for rd in rowdata:
        data.append(rd.text)
    if len(data)!=3:
        continue
    else:
        #skip unassigned postal codes
        if data[1]=="Not assigned":
            continue
        #if the neighborhood is not assigned, the borough is the neighborhood
        elif data[2]=="Not assigned\n":
            Hood.append(data[1])
            Boro.append(data[1])
            PC.append(data[0])
        else:
            Hood.append(data[2][:-1]) #removes the \n from the neighborhood text
            Boro.append(data[1])
            PC.append(data[0])
            

In [5]:
#dictionary of data for dataframe
dict_={'Postal Code':PC, 'Borough':Boro, 'Neighborhood':Hood}
df=pd.DataFrame.from_dict(dict_)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


# Final Data frame

In [7]:
df['Neighborhood']=df[['Postal Code', 'Borough', 'Neighborhood']].groupby(['Postal Code','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
df=df[['Postal Code', 'Borough', 'Neighborhood']].drop_duplicates()

## in the code above, I grouped the dataframe by postal code and borough and combined neighborhoods in identical boroughs

# Final DataFrame:

In [8]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor,Lawrence Heigh..."
6,M7A,Queen's Park,Queen's Park


In [9]:
df.shape

(103, 3)